In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
from tabulate import tabulate
import datetime
import os
import requests
import json
from pandas.io.json import json_normalize
from sqlalchemy import create_engine
import psycopg2
import psycopg2.extras
import mysql.connector
from mysql.connector import Error


In [16]:
# Reading data
accounts = pd.read_csv("accounts.csv")

inv_items = pd.read_csv("invoice_line_items.csv")

invoices = pd.read_csv("invoices.csv")

skus = pd.read_csv("skus.csv")

In [54]:
# Create the pandas DataFrame
df = pd.DataFrame(columns=['inv_id', 'acc_id', 'inv_total', 'inv_items', 'acct_age', 'num_inv_120d', 'cum_tot_inv_acct', 'is_late'])

In [55]:
#assigning first column of main table
df['inv_id'] = invoices['invoice_id']

In [56]:
#assigning second column of main table
df['acc_id'] = invoices['account_id']

In [57]:
#assigning third column of main table
df['inv_total'] = 0.00

a=0
for i in inv_items['invoice_id']: #iterating through the invoice_line_items table
    total = inv_items['quantity'].iat[a] * skus['item_cost_price'].iat[skus[skus['item_id'] == inv_items['item_id'].iat[a]].index[0]]
    
    df['inv_total'].iat[df[df['inv_id'] == i].index[0]] = (df['inv_total'].iat[df[df['inv_id'] == i].index[0]] + total)
    
    total = 0
    a = a + 1

In [58]:
#assigning forth column of main table
df['inv_items'] = 0

a=0
for i in inv_items['invoice_id']: #iterating through the invoice_line_items table
    total = inv_items['quantity'].iat[a]
    
    df['inv_items'].iat[df[df['inv_id'] == i].index[0]] = (df['inv_items'].iat[df[df['inv_id'] == i].index[0]] + total)
    
    total = 0
    a = a + 1

In [64]:
#assigning fifth column of main table
df['acct_age'] = 0

for i in range(0,len(invoices['invoice_id'])):
    diff = abs(datetime.datetime.strptime(invoices['date_issued'].iat[i],'%Y-%m-%d') - datetime.datetime.strptime(accounts['joining_date'].iat[accounts[accounts['account_id'] == invoices['account_id'].iat[i]].index[0]],'%d-%m-%y')).days
    
    df['acct_age'].iat[df[df['inv_id'] == invoices['invoice_id'].iat[i]].index[0]] = diff

In [65]:
#assigning sixth column of main table
df['num_inv_120d'] = 0

for i in range(0,len(invoices['invoice_id'])):
    for j in range(i,len(invoices['invoice_id'])):
        if invoices['account_id'].iat[i] == invoices['account_id'].iat[j]: 
            diff = abs(datetime.datetime.strptime(invoices['date_issued'].iat[i],'%Y-%m-%d') - datetime.datetime.strptime(invoices['date_issued'].iat[j],'%Y-%m-%d')).days
            if diff < 120:
                if invoices['date_issued'].iat[i] > invoices['date_issued'].iat[j]:
                    df['num_inv_120d'].iat[df[df['inv_id'] == invoices['invoice_id'].iat[i]].index[0]] = (df['num_inv_120d'].iat[df[df['inv_id'] == invoices['invoice_id'].iat[i]].index[0]] + 1)
                elif invoices['date_issued'].iat[i] < invoices['date_issued'].iat[j]:
                    df['num_inv_120d'].iat[df[df['inv_id'] == invoices['invoice_id'].iat[j]].index[0]] = (df['num_inv_120d'].iat[df[df['inv_id'] == invoices['invoice_id'].iat[j]].index[0]] + 1)

In [ ]:
#assigning seventh column of main table
df['cum_tot_inv_acct'] = 1

for i in range(0,len(invoices['invoice_id'])):
    for j in range(0,len(invoices['invoice_id'])):
        if invoices['account_id'].iat[i] == invoices['account_id'].iat[j] and invoices['date_issued'].iat[i] > invoices['date_issued'].iat[j]:
            
            df['cum_tot_inv_acct'].iat[df[df['inv_id'] == invoices['invoice_id'].iat[i]].index[0]] = (df['cum_tot_inv_acct'].iat[df[df['inv_id'] == invoices['invoice_id'].iat[i]].index[0]] + 1)


In [ ]:
#assigning eighth column of main table

for i in range(0,len(invoices['invoice_id'])):
    diff = abs(datetime.datetime.strptime(invoices['payment_dates'].iat[i],'%Y-%m-%d') - datetime.datetime.strptime(invoices['date_issued'].iat[i],'%Y-%m-%d')).days
    if diff > 30:
        df['is_late'].iat[df[df['inv_id'] == invoices['invoice_id'].iat[i]].index[0]] = 1
    else:
        df['is_late'].iat[df[df['inv_id'] == invoices['invoice_id'].iat[i]].index[0]] = 0

In [ ]:
#This is where you can create a csv file of final table. (Remove the # in next line)
#df.to_csv('Reece.csv', index=False)

In [ ]:
x = df[['inv_total','inv_items','acct_age','num_inv_120d','cum_tot_inv_acct']] #Contains all independent variables
y = df[['is_late']] #Contain dependent variable (is_late)

x=x.astype('int')
y=y.astype('int')

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state = 2)

In [ ]:
print("\nCreating classification model using K-Nearest Neighbour\n")

print("\nChecking for n_neighbour = 3")
classifier = KNeighborsClassifier(n_neighbors=3)
classifier.fit(x_train, y_train.values.ravel())

y_pred1 = classifier.predict(x_test)
print(confusion_matrix(y_test, y_pred1))
print(classification_report(y_test, y_pred1))
print(accuracy_score(y_test, y_pred1))

print("\nChecking for n_neighbour = 4")
classifier = KNeighborsClassifier(n_neighbors=4)
classifier.fit(x_train, y_train.values.ravel())

y_pred2 = classifier.predict(x_test)
print(confusion_matrix(y_test, y_pred2))
print(classification_report(y_test, y_pred2))
print(accuracy_score(y_test, y_pred2))

print("\nChecking for n_neighbour = 5")
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(x_train, y_train.values.ravel())

y_pred3 = classifier.predict(x_test)
print(confusion_matrix(y_test, y_pred3))
print(classification_report(y_test, y_pred3))
print(accuracy_score(y_test, y_pred3))

In [ ]:
print("\nCreating classification model using Decision Tree\n")

print("\nUsing splitter = 'best' and max_features = 'sqrt'")
classifier = DecisionTreeClassifier(splitter = 'best', max_features = 'sqrt')
classifier.fit(x_train, y_train.values.ravel())

y_pred1 = classifier.predict(x_test)

print(confusion_matrix(y_test, y_pred1))
print(classification_report(y_test, y_pred1))
print(accuracy_score(y_test, y_pred1))

print("\nUsing splitter = 'best' and max_features = 'log2'")
classifier = DecisionTreeClassifier(splitter = 'best', max_features = 'log2')
classifier.fit(x_train, y_train.values.ravel())

y_pred2 = classifier.predict(x_test)

print(confusion_matrix(y_test, y_pred2))
print(classification_report(y_test, y_pred2))
print(accuracy_score(y_test, y_pred2))

print("\nUsing splitter = 'random' and max_features = 'sqrt'")
classifier = DecisionTreeClassifier(splitter = 'random', max_features = 'sqrt')
classifier.fit(x_train, y_train.values.ravel())

y_pred3 = classifier.predict(x_test)

print(confusion_matrix(y_test, y_pred3))
print(classification_report(y_test, y_pred3))
print(accuracy_score(y_test, y_pred3))

print("\nUsing splitter = 'random' and max_features = 'log2'")
classifier = DecisionTreeClassifier(splitter='random', max_features = 'log2')
classifier.fit(x_train, y_train.values.ravel())

y_pred4 = classifier.predict(x_test)

print(confusion_matrix(y_test, y_pred4))
print(classification_report(y_test, y_pred4))
print(accuracy_score(y_test, y_pred4))

In [ ]:
print("\nCreating classification model using Random Forest")

print("\nPart A")
print("\nFor n_estimators = 50 and random state = 2")
classifier = RandomForestClassifier(n_estimators=50, random_state=2)
classifier.fit(x_train, y_train.values.ravel())

y_pred = classifier.predict(x_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

print("\nFor n_estimators = 20 and random state = 2")
classifier = RandomForestClassifier(n_estimators=20, random_state=2)
classifier.fit(x_train, y_train.values.ravel())

y_pred = classifier.predict(x_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

print("\nFor n_estimators = 20 and random state = 6")
classifier = RandomForestClassifier(n_estimators=20, random_state=6)
classifier.fit(x_train, y_train.values.ravel())

y_pred = classifier.predict(x_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

In [14]:
#Establishing connection to postgres and creating tables

hostname = 'localhost'
database = 'reece'
username = 'postgres'
pwd = 'admin'
port_id = 5432
conn = None

connection = psycopg2.connect(
                host = hostname,
                dbname = database,
                user = username,
                password = pwd,
                port = port_id)

            
cursor = connection.cursor()

#cursor.execute('DROP TABLE IF EXISTS reece')

create_script = ''' CREATE TABLE IF NOT EXISTS reece (
                                    inv_id      int PRIMARY KEY,
                                    acc_id      int,
                                    inv_total   float(10),
                                    inv_items    int,
                                    acct_age  int,
                                    num_inv_120d  int,
                                    cum_tot_inv_acct  int,
                                    is_late int) '''

cursor.execute(create_script)

create_script = ''' CREATE TABLE IF NOT EXISTS accounts (
                                    account_id      int PRIMARY KEY,
                                    company_name      Varchar(200),
                                    company_address  Varchar(200),
                                    contact_person    char(40),
                                    contact_phone  varchar(200),
                                    gender  char(7),
                                    joining_date    date) '''

cursor.execute(create_script)  

create_script = ''' CREATE TABLE IF NOT EXISTS invoices (
                                    invoice_id      int PRIMARY KEY,
                                    account_id      int,
                                    date_issued    date,
                                    payment_dates    date,
                                    FOREIGN KEY (account_id) REFERENCES accounts(account_id)) '''

cursor.execute(create_script)

create_script = ''' CREATE TABLE IF NOT EXISTS skus (
                                    item_id      varchar(20),
                                    item_name    varchar(200),
                                    item_description    varchar(200),
                                    item_cost_price    float(10),
                                    item_retail_price    float(10)) '''

cursor.execute(create_script)

create_script = ''' CREATE TABLE IF NOT EXISTS invoice_line_item (
                                    invoice_id      int,
                                    item_id      varchar(20),
                                    quantity    int,
                                    FOREIGN KEY (item_id) REFERENCES skus(item_id),
                                    FOREIGN KEY (invoice_id) REFERENCES invoices(invoice_id)) '''

cursor.execute(create_script)

connection.commit()

In [ ]:
#Inserting data in the tables
cursor = connection.cursor()

cols = ",".join([str(i) for i in df.columns.tolist()])

# Insert DataFrame records one by one.
for i,row in df.iterrows():
    try:
        sql = "INSERT INTO reece (" +cols + ") VALUES (" + "%s,"*(len(row)-1) + "%s)"
        cursor.execute(sql, tuple(row))

        # the connection is not autocommitted by default, so we must commit to save our changes
        connection.commit()

    except psycopg2.IntegrityError:
        print("Duplicate values found.  Insert was not successful")
        
    # the connection is not autocommitted by default, so we must commit to save our changes
    connection.commit()

a = 0
for i in accounts['joining_date']:
    accounts['joining_date'].iat[a] = datetime.datetime.strptime(accounts['joining_date'].iat[a], '%d-%m-%y')
    a = a + 1
      
cols = ",".join([str(i) for i in accounts.columns.tolist()])

# Insert DataFrame records one by one.
for i,row in accounts.iterrows():
    try:
        sql = "INSERT INTO accounts (" +cols + ") VALUES (" + "%s,"*(len(row)-1) + "%s)"
        cursor.execute(sql, tuple(row))

        # the connection is not autocommitted by default, so we must commit to save our changes
        connection.commit()

    except psycopg2.IntegrityError:
        print("Duplicate values found.  Insert was not successful")
        
    # the connection is not autocommitted by default, so we must commit to save our changes
    connection.commit()
    
cols = ",".join([str(i) for i in invoices.columns.tolist()])

# Insert DataFrame recrds one by one.
for i,row in invoices.iterrows():
    try:
        sql = "INSERT INTO invoices (" +cols + ") VALUES (" + "%s,"*(len(row)-1) + "%s)"
        cursor.execute(sql, tuple(row))

        # the connection is not autocommitted by default, so we must commit to save our changes
        connection.commit()

    except psycopg2.IntegrityError:
        print("Duplicate values found.  Insert was not successful")
        
    # the connection is not autocommitted by default, so we must commit to save our changes
    connection.commit()
    
cols = ",".join([str(i) for i in skus.columns.tolist()])

# Insert DataFrame recrds one by one.
for i,row in skus.iterrows():
    try:
        sql = "INSERT INTO skus (" +cols + ") VALUES (" + "%s,"*(len(row)-1) + "%s)"
        cursor.execute(sql, tuple(row))

        # the connection is not autocommitted by default, so we must commit to save our changes
        connection.commit()

    except psycopg2.IntegrityError:
        print("Duplicate values found.  Insert was not successful")
        
    # the connection is not autocommitted by default, so we must commit to save our changes
    connection.commit()
    
cols = ",".join([str(i) for i in inv_items.columns.tolist()])

# Insert DataFrame recrds one by one.
for i,row in inv_items.iterrows():
    try:
        sql = "INSERT INTO invoice_line_item (" +cols + ") VALUES (" + "%s,"*(len(row)-1) + "%s)"
        cursor.execute(sql, tuple(row))

        # the connection is not autocommitted by default, so we must commit to save our changes
        connection.commit()

    except psycopg2.IntegrityError:
        print("Duplicate values found.  Insert was not successful")
        
    # the connection is not autocommitted by default, so we must commit to save our changes
    connection.commit()

In [ ]:
#This is the section where you can write your SQL query and execute to get desired output.

# create connection
connection = psycopg2.connect(
                host = hostname,
                dbname = database,
                user = username,
                password = pwd,
                port = port_id)

# Create cursor
my_cursor = connection.cursor()

# Execute Query
my_cursor.execute("SELECT * from skus")

# Fetch the records
result = my_cursor.fetchall()

for i in result:
    print(i)

# Close the connection
connection.close()